In [1]:

import numpy as np
from scipy import ndimage
from skimage.morphology import skeletonize
import matplotlib.pyplot as plt
from numba import njit, prange
import networkx as nx
import rasterio
from rasterio.features import shapes
from shapely.geometry import LineString, MultiLineString, Polygon, MultiPolygon
import geopandas as gpd
import gc
from shapely.geometry import shape


In [19]:


#VO related functions

@njit(parallel=True)
def calculate_vo(image, radius, exaggeration):
    rows, cols = image.shape
    vo_image = np.zeros_like(image, dtype=np.float32)
    circular_mask = create_circular_mask(radius)

    total_pixels = rows * cols

    for idx in prange(total_pixels):
        i, j = idx // cols, idx % cols

        volume_above_topography = 0.0
        total_sphere_volume = 0.0

        base_height = image[i, j] * exaggeration

        for di in range(-radius, radius + 1):
            for dj in range(-radius, radius + 1):
                ni, nj = i + di, j + dj

                if 0 <= ni < rows and 0 <= nj < cols and circular_mask[di + radius, dj + radius]:
                    height_in_sphere = radius - np.sqrt(di**2 + dj**2)
                    height_above_base = base_height - (image[ni, nj] * exaggeration)
                    effective_height = min(height_in_sphere, height_above_base)

                    if effective_height > 0:
                        volume_above_topography += effective_height

                    total_sphere_volume += height_in_sphere

        if total_sphere_volume > 0:
            vo_image[i, j] = volume_above_topography / total_sphere_volume

    return vo_image

@njit
def create_circular_mask(radius):
    size = 2 * radius + 1
    mask = np.zeros((size, size), dtype=np.bool_)
    center = radius
    for i in range(size):
        for j in range(size):
            if (i - center) ** 2 + (j - center) ** 2 <= radius ** 2:
                mask[i, j] = True
    return mask

@njit
def threshold_vo(vo_image, threshold):
    thresholded_image = np.where(vo_image >= threshold, 1, 0)
    return thresholded_image

def fill_bedform_holes(image):
    filled_image = ndimage.binary_fill_holes(image).astype(int)
    return filled_image

# SKeleton creation and simplification related functions 

def skeleton_to_graph(skeleton):
    """Convert skeleton to graph."""
    G = nx.Graph()
    rows, cols = skeleton.shape
    for row in range(rows):
        for col in range(cols):
            if skeleton[row, col]:
                G.add_node((row, col))
                # Add edges to the 8-connected neighbors
                for d_row, d_col in [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]:
                    n_row, n_col = row + d_row, col + d_col
                    if 0 <= n_row < rows and 0 <= n_col < cols and skeleton[n_row, n_col]:
                        G.add_edge((row, col), (n_row, n_col))
    return G

def find_longest_paths(G):
    """Find the longest paths in each connected component of the graph."""
    longest_paths = []
    for component in nx.connected_components(G):
        subgraph = G.subgraph(component)
        nodes = list(subgraph.nodes)
        longest_path = []
        for node in nodes:
            lengths = nx.single_source_dijkstra_path_length(subgraph, node)
            farthest_node = max(lengths, key=lengths.get)
            path = nx.dijkstra_path(subgraph, node, farthest_node)
            if len(path) > len(longest_path):
                longest_path = path
        longest_paths.append(longest_path)
    return longest_paths

def convert_to_geographical_coords(longest_paths, transform):
    """Convert pixel coordinates of the longest paths to geographical coordinates."""
    geographical_paths = []
    for path in longest_paths:
        geo_path = []
        for row, col in path:
            x, y = rasterio.transform.xy(transform, row, col)
            geo_path.append((x, y))
        geographical_paths.append(geo_path)
    return geographical_paths

def export_paths_to_shapefile(geographical_paths, crs, output_file):
    """Export the longest paths as a shapefile."""
    line_geometries = [LineString(path) for path in geographical_paths if len(path) > 1]
    gdf = gpd.GeoDataFrame(geometry=line_geometries, crs=crs)
    gdf.to_file(output_file, driver='ESRI Shapefile')

    
def create_longest_paths_image(skeleton, longest_paths):
    """ Create a binary image with the longest paths marked """
    paths_image = np.zeros_like(skeleton)
    for longest_path in longest_paths:
        for (row, col) in longest_path:
            paths_image[row, col] = 1
    return paths_image


# Functions related to converting to shapefiles 


def load_binary_image(filepath):
    """Load a binary TIFF image using rasterio"""
    with rasterio.open(filepath) as main_ds:
        main_inarr = main_ds.read(1)  # Read the first band
        profile = main_ds.profile
    binary_image = main_inarr > 0  # Ensure the image is binary
    return binary_image, profile

def extract_polygons(binary_image, transform):
    """Extract polygons from a binary image"""
    binary_image = binary_image.astype(np.uint8)  # Convert to uint8
    results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) in enumerate(
            shapes(binary_image, mask=binary_image, transform=transform)
        )
    )
    polygons = [shape(feature['geometry']) for feature in results]
    return polygons

def save_polygons_to_geopackage(polygons, crs, output_filepath):
    """Save polygons to a GeoPackage"""
    gdf = gpd.GeoDataFrame(geometry=polygons, crs=crs)
    gdf.to_file(output_filepath, driver="GPKG")



def snakes_algorithm(polygon, alpha, beta, iterations=10, with_z=False):
    """
    Implémentation de l'algorithme Snakes pour lisser un polygone.

    Parameters:
        polygon (Polygon): Polygone à lisser.
        alpha (float): Paramètre de tension.
        beta (float): Paramètre de rigidité.
        iterations (int): Nombre d'itérations.
        with_z (bool): Si True, prend en compte les coordonnées Z.

    Returns:
        Polygon: Polygone lissé.
    """
    coords = np.array(polygon.exterior.coords)
    n = len(coords)

    # Gestion des coordonnées Z
    if with_z and coords.shape[1] == 3:
        x, y, z = coords[:, 0], coords[:, 1], coords[:, 2]
    else:
        x, y = coords[:, 0], coords[:, 1]
        z = None

    # Définition des paramètres
    a = 2.0 * alpha + 6.0 * beta
    b = -alpha - 4.0 * beta
    c = beta
    plus = 4

    # Crée une matrice g (n + 2 * plus) x (n + 2 * plus)
    size = n + 2 * plus
    g = np.zeros((size, size))
    val = [c, b, a, b, c]

    for i in range(size):
        for j in range(size):
            index = j - i + 2
            if 0 <= index < len(val):
                g[i, j] = val[index]

    # Ajouter une identité pour stabiliser la matrice
    g += np.eye(size)

    # Inverser la matrice
    g_inv = np.linalg.inv(g)

    # Créer des vecteurs pour x, y (et z si nécessaire)
    x_coord = np.zeros(size)
    y_coord = np.zeros(size)
    z_coord = np.zeros(size) if z is not None else None

    # Remplir les vecteurs avec des données initiales
    x_coord[plus:n + plus] = x - x[0]
    y_coord[plus:n + plus] = y - y[0]
    if z is not None:
        z_coord[plus:n + plus] = z - z[0]

    # Répéter les points en début et en fin pour gérer les bords
    x_coord[:plus] = x[:plus][::-1] - x[0]
    y_coord[:plus] = y[:plus][::-1] - y[0]
    if z is not None:
        z_coord[:plus] = z[:plus][::-1] - z[0]

    x_coord[n + plus:] = x[-plus:] - x[0]
    y_coord[n + plus:] = y[-plus:] - y[0]
    if z is not None:
        z_coord[n + plus:] = z[-plus:] - z[0]

    # Appliquer les itérations
    for _ in range(iterations):
        x_coord = np.dot(g_inv, x_coord)
        y_coord = np.dot(g_inv, y_coord)
        if z is not None:
            z_coord = np.dot(g_inv, z_coord)

    # Construire un nouveau polygone lissé
    x_out = x_coord[plus:n + plus] + x[0]
    y_out = y_coord[plus:n + plus] + y[0]
    if z is not None:
        z_out = z_coord[plus:n + plus] + z[0]
        smooth_coords = np.column_stack((x_out, y_out, z_out))
    else:
        smooth_coords = np.column_stack((x_out, y_out))

    return Polygon(smooth_coords)


# Appliquer sur un GeoDataFrame
def smooth_geodataframe(gdf, alpha, beta, iterations=10):
    def smooth_geometry(geometry):
        if geometry.geom_type == "Polygon":
            return snakes_algorithm(geometry, alpha, beta, iterations)
        elif geometry.geom_type == "MultiPolygon":
            return MultiPolygon([snakes_algorithm(poly, alpha, beta, iterations) for poly in geometry.geoms])
        else:
            return geometry
    
    gdf["geometry"] = gdf["geometry"].apply(smooth_geometry)
    return gdf


def smooth_linestring(linestring, alpha, beta, iterations=10):
    """
    Smooth a LineString using the Snakes algorithm.
    
    Parameters:
        linestring (LineString): Input LineString to smooth.
        alpha (float): Tension parameter (controls elasticity).
        beta (float): Rigidity parameter (controls stiffness).
        iterations (int): Number of smoothing iterations.
        
    Returns:
        LineString: Smoothed LineString.
    """
    coords = np.array(linestring.coords)  # Extract the coordinates
    n = len(coords)  # Number of points in the LineString

    # Check if the line has enough points
    if n < 3:
        return linestring  # Skip smoothing for very short lines

    # Define parameters for the smoothing algorithm
    a = 2.0 * alpha + 6.0 * beta
    b = -alpha - 4.0 * beta
    c = beta
    plus = min(4, n - 1)  # Adjust padding size for short lines

    # Create the smoothing matrix
    size = n + 2 * plus
    g = np.zeros((size, size))
    val = [c, b, a, b, c]

    for i in range(size):
        for j in range(size):
            index = j - i + 2
            if 0 <= index < len(val):
                g[i, j] = val[index]

    # Add identity to stabilize the matrix
    g += np.eye(size)

    # Invert the smoothing matrix
    g_inv = np.linalg.inv(g)

    # Create vectors for x and y coordinates
    x_coord = np.zeros(size)
    y_coord = np.zeros(size)

    # Fill the vectors with the original coordinates
    x_coord[plus:n + plus] = coords[:, 0] - coords[0, 0]
    y_coord[plus:n + plus] = coords[:, 1] - coords[0, 1]

    # Add padding dynamically based on available points
    x_coord[:plus] = coords[:plus][::-1, 0] - coords[0, 0]
    y_coord[:plus] = coords[:plus][::-1, 1] - coords[0, 1]
    x_coord[n + plus:] = coords[-plus:, 0] - coords[0, 0]
    y_coord[n + plus:] = coords[-plus:, 1] - coords[0, 1]

    # Apply smoothing iterations
    for _ in range(iterations):
        x_coord = np.dot(g_inv, x_coord)
        y_coord = np.dot(g_inv, y_coord)

    # Construct the smoothed LineString
    x_out = x_coord[plus:n + plus] + coords[0, 0]
    y_out = y_coord[plus:n + plus] + coords[0, 1]
    smoothed_coords = np.column_stack((x_out, y_out))

    return LineString(smoothed_coords)

def smooth_centerlines(gdf, alpha, beta, iterations=10):
    """
    Smooth the centerlines in a GeoDataFrame.
    
    Parameters:
        gdf (GeoDataFrame): GeoDataFrame containing LineString geometries.
        alpha (float): Tension parameter (controls elasticity).
        beta (float): Rigidity parameter (controls stiffness).
        iterations (int): Number of smoothing iterations.
        
    Returns:
        GeoDataFrame: GeoDataFrame with smoothed centerlines.
    """
    def smooth_geometry(geometry):
        if geometry.geom_type == "LineString":
            return smooth_linestring(geometry, alpha, beta, iterations)
        elif geometry.geom_type == "MultiLineString":
            return MultiLineString([smooth_linestring(line, alpha, beta, iterations) for line in geometry.geoms])
        else:
            return geometry  # Leave other geometry types unchanged

    gdf["geometry"] = gdf["geometry"].apply(smooth_geometry)
    return gdf

######################################################################################



In [7]:
# Paths for input and output files
Input_topo = r'material\keewatin.tif'  # Input topographic data
Input_ndwi = r'material\keewatin_ndwi.tif'  # Input NDWI (Normalized Difference Water Index) data

Output_VO = r'outputs\final_dot\keewatin_VO.tif'  # Output file for Volumetric Obscurance (VO)
Output_threshold = r'outputs\final_dot\keewatin_Thresholded.tif'  # Thresholded VO image
Output_filled_masked = r'outputs\final_dot\keewatin_dev_filled_masked_threshold.tif'  # Masked and filled threshold image
Output_bedforms_gpkg = r'outputs\final_dot\keewatin_bedforms.gpkg'  # Output file for bedform polygons
Output_bedforms_smoothed_gpkg = r'outputs\final_dot\keewatin_bedforms_smoothed.gpkg'

Output_skeleton = r'outputs\final_dot\keewatin_skeletons.tif'  # Skeletonized image
Output_simplified_skel_image = r'outputs\final_dot\keewatin_simplified_skeletons.tif'  # Simplified skeleton image
Output_simplified_skel_vectors = r'outputs\final_dot\keewatin_longest_paths.gpkg'  # Longest path shapefile
Output_smoothed_simplified_skel_vectors = r'outputs\final_dot\keewatin_centerlines_smoothed.gpkg'  # Smoothed bedform polygons

# Parameters
vo_threshold = 0.625  # Threshold value for Volumetric Obscurance
radius = 50  # Radius parameter for VO calculation
exaggeration = 1000  # Exaggeration factor for elevation
alpha, beta, iterations = 1, 1, 1  # Parameters for smoothing algorithm

print("Opening topographic data file...")

# Load the topographic data
with rasterio.open(Input_topo) as main_ds:
    main_inarr = main_ds.read(1)  # Read the first band of the raster
    profile = main_ds.profile  # Save metadata for output files

print("Computing Volumetric Obscurance (VO)...")
vo_image = calculate_vo(main_inarr, radius, exaggeration)  # Calculate VO using input parameters

print("Saving VO as a TIFF file...")
# Save the VO image as a GeoTIFF
with rasterio.open(Output_VO, 'w', **profile) as dst:
    dst.write(vo_image, 1)

# Clean up memory
del main_inarr, main_ds
gc.collect()

print("Applying threshold to VO image...")
thresholded_image = threshold_vo(vo_image, vo_threshold)  # Threshold the VO image

# Clean up memory
del vo_image
gc.collect()

print("Saving thresholded VO image as a TIFF file...")
# Save the thresholded VO image as a GeoTIFF
with rasterio.open(Output_threshold, 'w', **profile) as dst:
    dst.write(thresholded_image, 1)

# Clean up memory
del thresholded_image
gc.collect()

print("Opening NDWI data file...")
# Load NDWI data to create a mask
with rasterio.open(Input_ndwi) as ndwi_ds:
    ndwi_inarr = ndwi_ds.read(1)  # Read the first band of the raster

# Create a mask where NDWI is below 0
mask = ndwi_inarr < 0

# Clean up memory
del ndwi_inarr, ndwi_ds
gc.collect()

print("Opening thresholded VO file...")
# Reopen the thresholded VO image
with rasterio.open(Output_threshold) as main_ds:
    main_inarr = main_ds.read(1)  # Read the first band of the raster
    profile = main_ds.profile  # Save metadata for output files

print("Applying NDWI mask to thresholded VO image...")
thresholded_image = main_inarr * mask  # Apply the mask to the thresholded image

# Clean up memory
del main_inarr, mask
gc.collect()

print("Filling holes in the thresholded image...")
filled_image = fill_bedform_holes(thresholded_image)  # Fill holes in the image

# Clean up memory
del thresholded_image
gc.collect()

print("Saving filled and masked threshold image as a TIFF file...")
# Save the filled and masked image as a GeoTIFF
with rasterio.open(Output_filled_masked, 'w', **profile) as dst:
    dst.write(filled_image, 1)

print("Extracting polygons from filled image...")
# Extract polygons from the filled image
polygons = extract_polygons(filled_image, profile['transform'])



print("Saving extracted polygons to GeoPackage...")
save_polygons_to_geopackage(polygons, profile['crs'], Output_bedforms_gpkg)  # Save polygons as a GeoPackage

# -------------------------------------------
# Smoothing bedform polygons
# -------------------------------------------
print("Smoothing bedform polygons...")

# Convert the list of polygons to a GeoDataFrame
polygons_gdf = gpd.GeoDataFrame(geometry=polygons, crs=profile['crs'])

# Smooth the polygons using the provided function
bedforms = smooth_geodataframe(polygons_gdf, alpha, beta, iterations)

# Print success message
print("Bedform polygons smoothed successfully.")


print("Saving smoothed polygons to GeoPackage...")
bedforms.to_file(Output_bedforms_smoothed_gpkg, driver="GPKG")

# -------------------------------------------
# Skeletonization
# -------------------------------------------
print("Skeletonizing the filled image...")
skeleton_image = skeletonize(filled_image)  # Perform skeletonization

# Clean up memory
del filled_image
gc.collect()

print("Saving skeletonized image as a TIFF file...")
# Save the skeletonized image as a GeoTIFF
with rasterio.open(Output_skeleton, 'w', **profile) as dst:
    dst.write(skeleton_image, 1)

print("Opening skeleton image...")
# Reopen the skeletonized image
with rasterio.open(Output_skeleton) as main_ds:
    skeleton = main_ds.read(1)
    profile = main_ds.profile
    transform = main_ds.transform
    crs = main_ds.crs

print("Creating graph from skeleton...")
G = skeleton_to_graph(skeleton)  # Convert skeleton to a graph

print("Finding longest paths in the skeleton...")
longest_paths = find_longest_paths(G)  # Find the longest paths in the graph

# Clean up memory
del G
gc.collect()

print("Creating longest paths image...")
paths_image = create_longest_paths_image(skeleton, longest_paths)  # Visualize longest paths

# Clean up memory
del skeleton
gc.collect()

print("Saving longest paths image as a TIFF file...")
# Save the longest paths image as a GeoTIFF
with rasterio.open(Output_simplified_skel_image, 'w', **profile) as dst:
    dst.write(paths_image, 1)

# Clean up memory
del paths_image
gc.collect()

print("Converting paths to geographical coordinates...")
geo_paths = convert_to_geographical_coords(longest_paths, transform)  # Convert paths to geographic coordinates

print("Saving longest paths")
# Export longest paths to a shapefile

line_geometries = [LineString(path) for path in geo_paths if len(path) > 1]
gdf = gpd.GeoDataFrame(geometry=line_geometries, crs=profile['crs'])
gdf.to_file(Output_simplified_skel_vectors, driver="GPKG")



# Suppose `centerlines_gdf` contains the centerlines as a GeoDataFrame
centerlines = smooth_centerlines(centerlines, alpha, beta, iterations)

print("Saving smoothed centerlines to GeoPackage...")
centerlines.to_file(Output_smoothed_simplified_skel_vectors, driver="GPKG")

print("Smoothed centerlines saved successfully.")

Opening topographic data file...
Computing Volumetric Obscurance (VO)...
Saving VO as a TIFF file...
Applying threshold to VO image...
Saving thresholded VO image as a TIFF file...
Opening NDWI data file...
Opening thresholded VO file...
Applying NDWI mask to thresholded VO image...
Filling holes in the thresholded image...
Saving filled and masked threshold image as a TIFF file...
Extracting polygons from filled image...


NameError: name 'shape' is not defined